<a href="https://colab.research.google.com/github/wandb/edu/blob/main/mlops-001/lesson1/02_Split.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
<!--- @wandbcode{course-lesson1} -->

# Data preparation

<!--- @wandbcode{course-lesson1} -->

In this notebook we will prepare the data to later train our deep learning model. To do so,

- we will start a new W&B `run` and use our raw data artifact
- split the data and save the splits into a new W&B Artifact
- join information about the split with our EDA Table

In [1]:
import os, warnings
import wandb

import pandas as pd
from fastai.vision.all import *
from sklearn.model_selection import StratifiedGroupKFold

import params
warnings.filterwarnings('ignore')

In [2]:
run = wandb.init(project=params.WANDB_PROJECT, entity=params.ENTITY, job_type="data_split")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: hlo (gaiusgaius). Use `wandb login --relogin` to force relogin


Let's use artifact we previously saved to W&B (we're storing artifact names and other global parameters in `params`).

In [3]:
raw_data_at = run.use_artifact(f'{params.RAW_DATA_AT}:latest')
path = Path(raw_data_at.download())

wandb: Downloading large artifact bdd_simple_1k:latest, 813.77MB. 4007 files... 
wandb:   4007 of 4007 files downloaded.  
Done. 0:0:3.1


In [4]:
path.ls()

(#5) [Path('artifacts/bdd_simple_1k:v4/media'),Path('artifacts/bdd_simple_1k:v4/images'),Path('artifacts/bdd_simple_1k:v4/eda_table.table.json'),Path('artifacts/bdd_simple_1k:v4/labels'),Path('artifacts/bdd_simple_1k:v4/LICENSE.txt')]

To split data between training, testing and validation, we need file names, groups (derived from the file name) and target (here we use our rare class bicycle for stratification). We previously saved these columns to EDA table, so let's retrieve it from the table now. 

In [5]:
fnames = os.listdir(path/'images')
groups = [s.split('-')[0] for s in fnames]

In [6]:
#count the number of images per group
pd.Series(groups).value_counts()

a91b7555    26
00e9be89    25
9a888ffa     5
2dc4e4d2     3
771df318     2
            ..
6dcbd3f4     1
75b224f0     1
24bff1fe     1
00aad4a0     1
9e4f94cd     1
Name: count, Length: 929, dtype: int64

In [7]:
orig_eda_table = raw_data_at.get("eda_table")

wandb: Downloading large artifact bdd_simple_1k:latest, 813.77MB. 4007 files... 
wandb:   4007 of 4007 files downloaded.  
Done. 0:0:2.9


In [8]:
orig_eda_table

In [9]:
y = orig_eda_table.get_column('bicycle')

Now we will split the data into train (80%), validation (10%) and test (10%) sets. As we do that, we need to be careful to:

- *avoid leakage*: for that reason we are grouping data according to video identifier (we want to make sure our model can generalize to new cars or video frames)
- handle the *label imbalance*: for that reason we stratify data with our target column

We will use sklearn's `StratifiedGroupKFold` to split the data into 10 folds and assign 1 fold for test, 1 for validation and the rest for training.

In [10]:
df = pd.DataFrame()
df['File_Name'] = fnames
df['fold'] = -1

In [11]:
df

,File_Name,fold
0,3b76e313-a4f861d4.jpg,-1
1,c665137e-6fffaf45.jpg,-1
2,42041574-fb905492.jpg,-1
3,587e93da-9d2eef22.jpg,-1
4,a188e945-124b917e.jpg,-1
...,...,...
995,53af996f-00000000.jpg,-1
996,9b0bf112-5edbe682.jpg,-1
997,6e9110c1-9973f5ab.jpg,-1
998,43d4a202-ca890610.jpg,-1


In [12]:
cv = StratifiedGroupKFold(n_splits=10)
for i, (train_idxs, test_idxs) in enumerate(cv.split(fnames, y, groups)):
    df.loc[test_idxs, ['fold']] = i

In [13]:
df

,File_Name,fold
0,3b76e313-a4f861d4.jpg,6
1,c665137e-6fffaf45.jpg,7
2,42041574-fb905492.jpg,8
3,587e93da-9d2eef22.jpg,7
4,a188e945-124b917e.jpg,9
...,...,...
995,53af996f-00000000.jpg,1
996,9b0bf112-5edbe682.jpg,0
997,6e9110c1-9973f5ab.jpg,1
998,43d4a202-ca890610.jpg,5


In [14]:
df['Stage'] = 'train'
df.loc[df.fold == 0, ['Stage']] = 'test'
df.loc[df.fold == 1, ['Stage']] = 'valid'
del df['fold']
df.Stage.value_counts()

Stage
train    800
valid    100
test     100
Name: count, dtype: int64

In [15]:
df.to_csv('data_split.csv', index=False)

We will now create a new artifact and add our data there. 

In [16]:
processed_data_at = wandb.Artifact(params.PROCESSED_DATA_AT, type="split_data")

In [17]:
processed_data_at.add_file('data_split.csv')
processed_data_at.add_dir(path)

wandb: Adding directory to artifact (./artifacts/bdd_simple_1k:v4)... Done. 9.1s


Finally, the split information may be relevant for our analyses - rather than uploading images again, we will save the split information to a new table and join it with EDA table we created previously. 

In [18]:
data_split_table = wandb.Table(dataframe=df[['File_Name', 'Stage']])

In [19]:
join_table = wandb.JoinedTable(orig_eda_table, data_split_table, "File_Name")

Let's add it to our artifact, log it and finish our `run`. 

In [20]:
processed_data_at.add(join_table, "eda_table_data_split")

ArtifactManifestEntry(path='eda_table_data_split.joined-table.json', digest='WPaSeqEOLBWQygUQbDKIpA==', ref=None, birth_artifact_id=None, size=127, extra={}, local_path='/home/hector/.local/share/wandb/artifacts/staging/tmp1r10o0pp')

In [21]:
run.log_artifact(processed_data_at)

In [22]:
run.finish()